# 인스타그램 크롤링

In [26]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time


driver = webdriver.Chrome('/Users/donguri/Workspace/데이터청년캠퍼스/wc/chromedriver')
driver.get('https://www.instagram.com')

In [27]:
keyword = input('검색어를 입력해주세요 :')
url = 'https://www.instagram.com/explore/tags/' + keyword
num_of_scroll = int((input('스크롤 횟수를 정해주세요 : ')))
driver.get(url)

url_list = []

for i in range(num_of_scroll):
    time.sleep(1.5)
    driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
    post_list = driver.find_elements_by_class_name('v1Nh3.kIKUG._bz0w')
    for post in post_list:
        url = post.find_element_by_css_selector('a').get_attribute('href')
        url_list.append(url)
        
url_list = list(set(url_list))
print(url_list)

검색어를 입력해주세요 : 빅데이터
스크롤 횟수를 정해주세요 :  3


['https://www.instagram.com/p/COb9VGsBdLI/', 'https://www.instagram.com/p/CRN0dEBjtJh/', 'https://www.instagram.com/p/CRP4yippGTn/', 'https://www.instagram.com/p/CROC2zKh83U/', 'https://www.instagram.com/p/CRO1O6rBNLJ/', 'https://www.instagram.com/p/CRPuipRDx4Q/', 'https://www.instagram.com/p/CRN9mPkplRD/', 'https://www.instagram.com/p/CROGURcssmL/', 'https://www.instagram.com/p/CPaI3LXgm6V/', 'https://www.instagram.com/p/CRJlk0zHhQ0/', 'https://www.instagram.com/p/CFEoNYvFkYA/', 'https://www.instagram.com/p/COMMBo0DMiK/', 'https://www.instagram.com/p/CRPj0imrLSH/', 'https://www.instagram.com/p/CRPjveAnto9/', 'https://www.instagram.com/p/CRNxx6QMeNR/', 'https://www.instagram.com/p/CRPyAeOFWz8/', 'https://www.instagram.com/p/CRN8_i8nWfa/', 'https://www.instagram.com/p/CKkUEg5nqNy/', 'https://www.instagram.com/p/CLeRfQRJ7VA/', 'https://www.instagram.com/p/CROCHVlFfBe/', 'https://www.instagram.com/p/CRP7CwGBd_9/', 'https://www.instagram.com/p/CROPH-lHhEu/', 'https://www.instagram.com/p/CR

In [32]:
import re
char = re.compile('[^ 0-9a-zA-Zr-ㅣ가-힣!?#.]')

for url in url_list:
    driver.get(url)
    insta_id = driver.find_element_by_class_name('e1e1d').text
    hash_text_area = driver.find_element_by_class_name('EtaWk')
    hash_text = hash_text_area.text
    
    hash_text = char.sub('', hash_text)
    heart = ""
    try:
        heart = driver.find_element_by_class_name('Nm9Fw').text
        heart = heart.split(' ')[-1][:-1].replace(",","")
    except:
        heart = "0"
    date = driver.find_element_by_class_name('k_Q0X.NnvRN').text
    published_date = ""
    
    if "시간 전 " in date:
        import datetime
        now = datetime.datetime.now()
        hours = date.split('시간 전')[0]
        published_date = now - datetime.timedelta(hours=int(hours))
        published_date = published_date.strftime('%m월 %일'.encode('unicode-escape').decode()).encode().decode('unicode-escape')
        
    elif "일 전" in date:
        import datetime
        now = datetime.datetime.now()
        days = date.split('일 전')[0]
        published_date = now - datetime.timedelta(days=int(days))
        published_date = published_date.strftime('%m월 %일'.encode('unicode-escape').decode()).encode().decode('unicode-escape')
    
    else:
        pass

# G마켓 베스트 상품 크롤링

In [3]:
!pip install io

ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io


In [6]:
from selenium import webdriver
import time
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 엑셀 저장을 위한 준비
import xlsxwriter
from datetime import datetime
import requests
from io import BytesIO

today = datetime.today().strftime("%Y%m%d")

file_name = 'gmarket.xlsx'
workbook = xlsxwriter.Workbook(file_name)
ws = workbook.add_worksheet()
ws.write(0,0,'상품 이미지')
ws.write(0,1,'상품 명')
ws.write(0,2,'상품 원가')
ws.write(0,3,'상품 할인가')
ws.write(0,4,'상품 할인률')
#행 열 크기 지정
ws.set_column(0,0,100)
ws.set_column(1,1,100)
ws.set_default_row(100)



driver = webdriver.Chrome('/Users/donguri/Workspace/데이터청년캠퍼스/wc/chromedriver')
driver.get('http://corners.gmarket.co.kr/Bestsellers')
time.sleep(1)

wait = WebDriverWait(driver, 10)
for i in range(30):
    driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
wait_state = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "img")))
best_list = driver.find_elements_by_css_selector('div.best-list')[1].find_elements_by_css_selector('ul > li')
count = 1

for item in best_list:
    thumb = item.find_element_by_class_name('thumb')
    thumb_img = thumb.find_element_by_css_selector('img.lazy').get_attribute('src')
    item_name = item.find_element_by_class_name('itemname').text
    item_price = item.find_element_by_class_name('item_price')
    o_price = item_price.find_element_by_class_name('o-price').text
    s_price = item_price.find_element_by_class_name('s-price').text

    try:
        sale = s_price.split(' ')[1]
    except:
        sale = '0%'
    s_price = s_price.split(' ')[0]
    if o_price == "":
        o_price = s_price
    
    ws.write(count, 1, item_name)
    ws.write(count, 2, o_price)
    ws.write(count, 3, s_price)
    ws.write(count, 4, sale)
    
    
    try:
        res = requests.get(thumb_img)
        image_data = BytesIO(res.content)
        image_size = len(image_data.getvalue())
        if image_size > 0:
            cell = 'A'+str(count)
            ws.insert_image(cell, thumb_img, {'image_data':image_data, 'x_scale':0.3, 'y_scale':0.3})
    except:
        pass
    
    count += 1
    
workbook.close()
print('수집 종료')    

수집 종료


# 네이버 뉴스 웹 크롤링

In [19]:
from selenium import webdriver
import time
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


keyword = input('키워드 입력 :')
driver = webdriver.Chrome('/Users/donguri/Workspace/데이터청년캠퍼스/wc/chromedriver')
driver.get('https://search.naver.com/search.naver?where=news&sm=tab_jum&query='+keyword)

now_page = driver.find_element_by_css_selector('div.sc_page > div.sc_page_inner > a').get_attribute('href')

url_list = []


cnt = 0

while True:
    cnt += 1
    if cnt >= 10:
        break
        
    time.sleep(1)
    news_group = driver.find_element_by_css_selector('div.group_news > ul.list_news')
    news_list = news_group.find_elements_by_css_selector('li')
    
    for news in news_list:
        url = news.find_element_by_css_selector('a').get_attribute('href')
        url_list.append(url)
    
    time.sleep(1)
    next_page = driver.find_element_by_css_selector('div.sc_page > a.btn_next').get_attribute('href')
    driver.get(next_page)

    
url_list = list(set(url_list))
print(url_list)

키워드 입력 : 경제


['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=421&aid=0005474480', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=421&aid=0005471282', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0012520626', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=011&aid=0003935197', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=417&aid=0000713872', 'http://www.newsway.co.kr/news/view?tp=1&ud=2021071216505218567', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0010603974', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=001&aid=0012521158', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0012520638', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0010603813', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0012517806', 'https://search.naver.com/search.naver?wher

In [20]:
len(url_list)

89

In [25]:
from selenium import webdriver
import time

driver = webdriver.Chrome('/Users/donguri/Workspace/데이터청년캠퍼스/wc/chromedriver')
def NaverNewsGetDate(driver, url):
    driver.get(url)
    content = driver.find_element_by_class_name('content')
    article_info = driver.find_element_by_class_name('article_info')
    article_title = article_info.find_element_by_id('articleTitle')
        
        
    title = article_title.text
    sponsor = article_info.find_element_by_class_name('sponsor')
    date = sponsor.find_element_class_name('t11').text
        
        
    article_body = content.find_element_by_class_name('article_body')
    article_content = article_body.find_element_by_id('articleBodyContents')
        
    content = article_content.text
        
    return title, date, content
    

for url in url_list:
    if 'news.naver.com' in url:
        try:
            title, date,content = NaverNewsGetDate(driver, url)
            data = {'title':title, 'date':date, 'content':content}
        except:
            pass
    elif 'app.yonhapnews.co.kr' in url:
        print('연합신문')
            
    elif 'news.hankyung.com' in url:
        print('한경신문')
            